In diesem Notebook werden auf verschiedenen Modalitäten verschiedene Modelle trainiert, angewendet und ausgewertet. Jeweils: Random Forest, XGBoost, Logistic Regression, SVM, Neuronales Netz

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Vorbereitung: Data Split into 3 Datasets!

In [17]:
df = pd.read_csv('features_enriched_tab_img_text.csv')
print(f"Orginal:\t{df.shape}")
print("-----------------------------------------------------------------------")

train, test = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=1)
train_val, test = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=1)
train, val = train_test_split(train_val, test_size=0.2, stratify=train_val['label'], random_state=1)

print(f"Train:\t\t{train.shape}")
print(f"Validation:\t{val.shape}")
print(f"Test:\t\t{test.shape}")

train.to_csv('detection_train.csv', index=False)
val.to_csv('detection_val.csv', index=False)
test.to_csv('detection_test.csv', index=False)

Orginal:	(1408, 562)
------------------------------------------------------------
Train:		(957, 562)
Validation:	(169, 562)
Test:		(282, 562)


# Text

# Tabular

# Image

# Multimodal